# AD2C : Test Bed

This Jupter Notebook aims to Experiment with different Component of AD2C framework Like task, ESC, Callback and Loggers. This will also include advance experimental setup to test the trainied model. 


----


## Imports

In [7]:
import sys
import os
import hydra
import wandb
import sys
import hydra
import time

from hydra.core.hydra_config import HydraConfig
from hydra.core.global_hydra import GlobalHydra
from omegaconf import DictConfig, OmegaConf

# Benchmarl & Project Imports
import benchmarl.models
from benchmarl.algorithms import *
from benchmarl.environments import VmasTask
from benchmarl.experiment import Experiment
from benchmarl.hydra_config import (
    load_algorithm_config_from_hydra,
    load_experiment_config_from_hydra,
    load_task_config_from_hydra,
    load_model_config_from_hydra,
)

# Custom Callbacks
from het_control.callback import *
from het_control.environments.vmas import render_callback
from het_control.models.het_control_mlp_empirical import HetControlMlpEmpiricalConfig
from het_control.callbacks.esc_callback import ExtremumSeekingController
from het_control.callbacks.sndESLogger import TrajectorySNDLoggerCallback

import numpy as np
import torch
import matplotlib.pyplot as plt
import networkx as nx
import wandb
from tensordict import TensorDict
from typing import List
from benchmarl.experiment.callback import Callback

## Wandb Fix
---

In [8]:
unique_id = f"AD2C_Eval_{int(time.time())}"

if not hasattr(wandb, "_custom_orig_init"):
    print("Saving original WandB init function...")
    wandb._custom_orig_init = wandb.init

def forced_wandb_init(*args, **kwargs):
    print(f"\n--- INTERCEPTING WANDB INIT ---")
    
    # Force the new ID and Name
    kwargs['id'] = unique_id
    kwargs['name'] = unique_id
    
    # Force "New Run" behavior
    kwargs['resume'] = "allow" 
    kwargs['reinit'] = True
    
    print(f"Forced ID: {unique_id}")
    print(f"-------------------------------\n")
    
    # We always call the SAVED original function, not the current one
    return wandb._custom_orig_init(*args, **kwargs)

# Apply the patch
wandb.init = forced_wandb_init


## SND Ploting Callback
This allow us to display plots for the snd. from the eval run. 

---

In [9]:
import numpy as np
import matplotlib.pyplot as plt
import wandb
import torch
import networkx as nx  # Required for the Graph Visualizer

class SNDHeatmapVisualizer:
    def __init__(self, key_name="Visuals/SND_Heatmap"):
        self.key_name = key_name

    def generate(self, snd_matrix, step_count):
        # snd_matrix is now GUARANTEED to be a clean 2D Numpy array
        n_agents = snd_matrix.shape[0]
        agent_labels = [f"Agent {i+1}" for i in range(n_agents)]
        
        # Calculate SND value
        iu = np.triu_indices(n_agents, k=1)
        if len(iu[0]) > 0:
            snd_value = float(np.mean(snd_matrix[iu]))
        else:
            snd_value = 0.0

        fig, ax = plt.subplots(figsize=(6, 5))

        im = ax.imshow(
            snd_matrix,
            cmap="viridis",
            interpolation="nearest",
            vmin=0, vmax=3 
        )

        ax.set_title(f"SND: {snd_value:.3f}  –  Step {step_count}")

        ax.set_xticks(np.arange(n_agents))
        ax.set_yticks(np.arange(n_agents))
        ax.set_xticklabels(agent_labels)
        ax.set_yticklabels(agent_labels)
        plt.setp(ax.get_xticklabels(), rotation=30, ha="right")

        fig.colorbar(im, ax=ax, label="Distance")

        for i in range(n_agents):
            for j in range(n_agents):
                val = snd_matrix[i, j]
                # Dynamic text color for visibility
                text_color = "white" if val < 1.0 else "black"
                ax.text(
                    j, i, f"{val:.2f}",
                    ha="center", va="center",
                    color=text_color,
                    fontsize=9, fontweight="bold"
                )

        plt.tight_layout()
        img = wandb.Image(fig)
        plt.close(fig)
        return {self.key_name: img}


class SNDBarChartVisualizer:
    def __init__(self, key_name="Visuals/SND_BarChart"):
        self.key_name = key_name

    def generate(self, snd_matrix, step_count):
        n_agents = snd_matrix.shape[0]
        
        # Create pairs i < j
        pairs = [(i, j) for i in range(n_agents) for j in range(i + 1, n_agents)]
        if not pairs:
            return {}

        pair_values = [float(snd_matrix[i, j]) for i, j in pairs]
        pair_labels = [f"A{i+1}-A{j+1}" for i, j in pairs]

        snd_value = float(np.mean(pair_values))

        fig, ax = plt.subplots(figsize=(8, 5))
        bars = ax.bar(pair_labels, pair_values, color="teal")

        ax.set_title(f"SND: {snd_value:.3f}  –  Step {step_count}")
        ax.set_ylabel("Distance")
        ax.set_ylim(0, 3)
        ax.tick_params(axis="x", rotation=45)

        ax.bar_label(bars, fmt="%.2f", padding=3)

        plt.tight_layout()
        img = wandb.Image(fig)
        plt.close(fig)
        return {self.key_name: img}


class SNDGraphVisualizer:
    def __init__(self, key_name="Visuals/SND_NetworkGraph"):
        self.key_name = key_name

    def generate(self, snd_matrix, step_count):
        n_agents = snd_matrix.shape[0]

        pairs = [(i, j) for i in range(n_agents) for j in range(i + 1, n_agents)]
        if not pairs:
            return {}

        pair_values = [float(snd_matrix[i, j]) for i, j in pairs]
        snd_value = float(np.mean(pair_values))

        fig = plt.figure(figsize=(7, 7))
        G = nx.Graph()

        for i, j in pairs:
            G.add_edge(i, j, weight=float(snd_matrix[i, j]))

        pos = nx.spring_layout(G, seed=42)
        weights = [G[u][v]['weight'] for u, v in G.edges()]

        nx.draw_networkx_nodes(G, pos, node_size=750, node_color='lightblue')
        
        label_mapping = {i: f"A{i+1}" for i in range(n_agents)}
        nx.draw_networkx_labels(G, pos, labels=label_mapping, font_size=12, font_weight='bold')

        edges = nx.draw_networkx_edges(
            G, pos,
            edge_color=weights,
            edge_cmap=plt.cm.viridis,
            width=2,
            edge_vmin=0, edge_vmax=3
        )

        edge_labels = {(i, j): f"{snd_matrix[i, j]:.2f}" for i, j in pairs}
        nx.draw_networkx_edge_labels(
            G, pos, edge_labels=edge_labels,
            font_color='black', font_size=9, font_weight='bold'
        )

        plt.colorbar(edges, label='Distance')
        plt.title(f"SND: {snd_value:.3f}  –  Step {step_count}", fontsize=14)
        plt.axis('off')

        img = wandb.Image(fig)
        plt.close(fig)
        return {self.key_name: img}


class SNDVisualizationManager:
    """
    Manages the individual visualizers and handles ALL data cleaning centrally.
    """
    def __init__(self):
        self.visualizers = [
            SNDHeatmapVisualizer(),
            SNDBarChartVisualizer(),
            SNDGraphVisualizer()
        ]

    def _prepare_matrix(self, snd_matrix):
        """
        Robustly converts and reshapes matrix.
        Fixes crash by Symmetrizing (Broadcasting) BEFORE accessing diagonals.
        """
        # 1. Convert to Numpy
        if hasattr(snd_matrix, "detach"):
            snd_matrix = snd_matrix.detach().cpu().numpy()
        elif not isinstance(snd_matrix, np.ndarray):
            snd_matrix = np.array(snd_matrix)

        # 2. "Peel" dimensions until we hit 2D
        # This turns (1, 2, 2) -> (2, 2) and (1, 2, 1) -> (2, 1)
        while snd_matrix.ndim > 2:
            snd_matrix = snd_matrix[0]

        # 3. Handle 1D edge case (if squeeze happened upstream)
        if snd_matrix.ndim == 1:
            # Try to reshape to square, or expand dims
            size = snd_matrix.shape[0]
            n_agents = int(np.sqrt(size))
            if n_agents * n_agents == size:
                snd_matrix = snd_matrix.reshape(n_agents, n_agents)
            else:
                # Treat as column vector (N, 1)
                snd_matrix = snd_matrix[:, None]

        # 4. Create copy
        snd_matrix = snd_matrix.copy()

        # 5. FIX: Enforce Symmetry FIRST
        # If input is (2, 1), this line broadcasts it: (2, 1) + (1, 2) = (2, 2)
        # This automatically "expands" the missing dimension.
        snd_matrix = (snd_matrix + snd_matrix.T) / 2.0

        # 6. NOW set diagonals (Safe because matrix is guaranteed square now)
        n = snd_matrix.shape[0]
        if n > 0:
            for i in range(n):
                snd_matrix[i, i] = 0.0
        
        return snd_matrix

    def generate_all(self, snd_matrix, step_count):
        # Clean the matrix ONCE here
        clean_matrix = self._prepare_matrix(snd_matrix)
        
        all_plots = {}
        for visualizer in self.visualizers:
            try:
                # Pass the clean matrix to all visualizers
                plots = visualizer.generate(clean_matrix, step_count)
                all_plots.update(plots)
            except Exception as e:
                print(f"Error generating {visualizer.__class__.__name__}: {e}")
                # Optional: Print shape to help debug if it fails again
                print(f"Failed Matrix Shape: {clean_matrix.shape}")
        return all_plots

In [10]:
class SNDVisualizerCallback(Callback):
    """
    Computes the SND matrix and uses the Manager to log visualizations.
    """
    def __init__(self):
        super().__init__()
        self.control_group = None
        self.model = None
        # Initialize the manager that holds the 3 plot classes
        self.viz_manager = SNDVisualizationManager()

    def on_setup(self):
        """Auto-detects the agent group and initializes the model wrapper."""
        if not self.experiment.group_policies:
            print("\nWARNING: No group policies found. SND Visualizer disabled.\n")
            return

        self.control_group = list(self.experiment.group_policies.keys())[0]
        policy = self.experiment.group_policies[self.control_group]
        
        # Ensure 'get_het_model' is imported or available in this scope
        self.model = get_het_model(policy)

        if self.model is None:
             print(f"\nWARNING: Could not extract HetModel for group '{self.control_group}'. Visualizer disabled.\n")

    def _get_agent_actions_for_rollout(self, rollout):
        """Helper to run the forward pass and get actions for SND computation."""
        obs = rollout.get((self.control_group, "observation"))
        actions = []
        for i in range(self.model.n_agents):
            temp_td = TensorDict(
                {(self.control_group, "observation"): obs},
                batch_size=obs.shape[:-1]
            )
            action_td = self.model._forward(temp_td, agent_index=i, compute_estimate=False)
            actions.append(action_td.get(self.model.out_key))
        return actions

    def on_evaluation_end(self, rollouts: List[TensorDict]):
        """Runs at the end of evaluation to compute SND and log plots."""
        if self.model is None:
            return

        logs_to_push = {}
        first_rollout_snd_matrix = None

        with torch.no_grad():
            for i, r in enumerate(rollouts):
                # We only need the matrix from the first rollout for clean visualization
                if i > 0: 
                    break

                agent_actions = self._get_agent_actions_for_rollout(r)
                
                # Ensure 'compute_behavioral_distance' is imported/available
                pairwise_distances_tensor = compute_behavioral_distance(agent_actions, just_mean=False)
                
                if pairwise_distances_tensor.ndim > 2:
                    pairwise_distances_tensor = pairwise_distances_tensor.mean(dim=0)

                first_rollout_snd_matrix = pairwise_distances_tensor.cpu().numpy()

        # Generate and Log Visualizations via the Manager
        if first_rollout_snd_matrix is not None:
            visual_logs = self.viz_manager.generate_all(
                snd_matrix=first_rollout_snd_matrix, 
                step_count=self.experiment.n_iters_performed
            )
            logs_to_push.update(visual_logs)
            
            # Update the logger
            self.experiment.logger.log(logs_to_push, step=self.experiment.n_iters_performed)

## Env Setup

This code block compiles all the different section of MARl env together to Run the experiment

---

In [11]:
# 1. EXPERIMENT LOGIC

def setup(task_name):
    benchmarl.models.model_config_registry.update(
        {
            "hetcontrolmlpempirical": HetControlMlpEmpiricalConfig,
        }
    )
    if task_name == "vmas/navigation":
        # Set the render callback for the navigation case study
        VmasTask.render_callback = render_callback

def get_experiment(cfg: DictConfig) -> Experiment:
    hydra_choices = HydraConfig.get().runtime.choices
    task_name = hydra_choices.task
    algorithm_name = hydra_choices.algorithm

    setup(task_name)

    print(f"\nAlgorithm: {algorithm_name}, Task: {task_name}")
    # print("\nLoaded config:\n") # Optional: Commented out to reduce clutter
    # print(OmegaConf.to_yaml(cfg))

    algorithm_config = load_algorithm_config_from_hydra(cfg.algorithm)
    experiment_config = load_experiment_config_from_hydra(cfg.experiment)
    task_config = load_task_config_from_hydra(cfg.task, task_name)
    critic_model_config = load_model_config_from_hydra(cfg.critic_model)
    model_config = load_model_config_from_hydra(cfg.model)

    if isinstance(algorithm_config, (MappoConfig, IppoConfig, MasacConfig, IsacConfig)):
        model_config.probabilistic = True
        model_config.scale_mapping = algorithm_config.scale_mapping
        algorithm_config.scale_mapping = (
            "relu"  # The scaling of std_dev will be done in the model
        )
    else:
        model_config.probabilistic = False

    experiment = Experiment(
        task=task_config,
        algorithm_config=algorithm_config,
        model_config=model_config,
        critic_model_config=critic_model_config,
        seed=cfg.seed,
        config=experiment_config,
        callbacks=[
            SndCallback(),
            ExtremumSeekingController(
                        control_group="agents",
                        initial_snd=0.0,
                        dither_magnitude=0.2,
                        dither_frequency_rad_s=1.0,
                        integral_gain=-0.1,
                        high_pass_cutoff_rad_s=1.0,
                        low_pass_cutoff_rad_s=1.0,
                        sampling_period=1.0
            ),
            SNDVisualizerCallback(),
            # TrajectorySNDLoggerCallback(control_group="agents"),
            NormLoggerCallback(),
            ActionSpaceLoss(
                use_action_loss=cfg.use_action_loss, action_loss_lr=cfg.action_loss_lr
            ),
        ]
        + (
            [
                TagCurriculum(
                    cfg.simple_tag_freeze_policy_after_frames,
                    cfg.simple_tag_freeze_policy,
                )
            ]
            if task_name == "vmas/simple_tag"
            else []
        ),
    )
    return experiment

## Training Code

Trains the model for 100 episodes. 

In [ ]:
ABS_CONFIG_PATH = "/home/grad/doc/2027/spatel2/AD2C_testBed/AD2C/ControllingBehavioralDiversity/het_control/conf"
CONFIG_NAME = "navigation_ippo"  # Make sure 'navigation_ippo.yaml' exists in the folder above!
SAVE_PATH = "/home/grad/doc/2027/spatel2/AD2C_testBed/model_checkpoints/navigation_ippo_esc/"

save_interval = 600000
desired_snd = 1.0
max_frame = 6000000

if not os.path.exists(SAVE_PATH):
    print(f"Creating missing directory: {SAVE_PATH}")
    os.makedirs(SAVE_PATH, exist_ok=True)

GlobalHydra.instance().clear()

sys.argv = [
    "dummy.py",
    f"model.desired_snd={desired_snd}",
    f"experiment.max_n_frames={max_frame}",
    f"experiment.checkpoint_interval={save_interval}",
    f"experiment.save_folder={SAVE_PATH}", 
    f"task.n_agents=2",
]

# 3. Define the Hydra wrapper
@hydra.main(version_base=None, config_path=ABS_CONFIG_PATH, config_name=CONFIG_NAME)
def hydra_experiment(cfg: DictConfig) -> None:
    print(f"Config loaded from: {ABS_CONFIG_PATH}")
    if wandb.run is not None:
        print("Finishing previous WandB run...")
        wandb.finish()
    
    print(f"Running with SND: {cfg.model.desired_snd}")
    
    experiment = get_experiment(cfg=cfg)
    experiment.run()
    wandb.finish()

# 4. Execute safely
if __name__ == "__main__":
    try:
        hydra_experiment()
    except SystemExit:
        print("Experiment finished successfully.")
    except Exception as e:
        print(f"An error occurred: {e}")

Config loaded from: /home/grad/doc/2027/spatel2/AD2C_testBed/AD2C/ControllingBehavioralDiversity/het_control/conf
Running with SND: 1.0

Algorithm: ippo, Task: vmas/navigation


Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
Users of this version of Gym should be able to simply replace 'import gym' with 'import gymnasium as gym' in the vast majority of cases.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.
/home/grad/doc/2027/spatel2/miniconda3/envs/ad2c/lib/python3.9/site-packages/torch/nn/functional.py:1956: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")



--- INTERCEPTING WANDB INIT ---
Forced ID: AD2C_Eval_1764920995
-------------------------------



wandb: Currently logged in as: svarp (svarp-university-of-massachusetts-lowell) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.



✅ SUCCESS: Extremum Seeking Controller initialized for group 'agents'.


...

mean return = 0.39752763509750366:  78%|███████▊  | 78/100 [39:43<11:20, 30.92s/it]/home/grad/doc/2027/spatel2/miniconda3/envs/ad2c/lib/python3.9/site-packages/torch/nn/functional.py:1956: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


[ESC] Updated SND: 0.6606975793838501 (Reward: 0.831, Update Step: -0.0844)


mean return = 0.5141802430152893:  79%|███████▉  | 79/100 [40:13<10:46, 30.78s/it] /home/grad/doc/2027/spatel2/miniconda3/envs/ad2c/lib/python3.9/site-packages/torch/nn/functional.py:1956: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


[ESC] Updated SND: 0.47815582156181335 (Reward: 0.564, Update Step: -0.1825)


mean return = 0.49059024453163147:  80%|████████  | 80/100 [40:44<10:15, 30.78s/it]/home/grad/doc/2027/spatel2/miniconda3/envs/ad2c/lib/python3.9/site-packages/torch/nn/functional.py:1956: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


## Eval Run
---
Single Step eval. from the check point

In [ ]:

wandb.init = forced_wandb_init

# --- STEP 1: Define Unique ID ---
unique_id = f"AD2C_Eval_{int(time.time())}"

# --- STEP 2: Force WandB to use this exact run ---
os.environ["WANDB_MODE"] = "online"
os.environ["WANDB_RUN_ID"] = unique_id
os.environ["WANDB_NAME"] = unique_id
os.environ["WANDB_INIT_TIMEOUT"] = "300"
os.environ["WANDB_SILENT"] = "true"

# --- STEP 3: Hydra Setup ---
ABS_CONFIG_PATH = "/home/grad/doc/2027/spatel2/AD2C_testBed/AD2C/ControllingBehavioralDiversity/het_control/conf"
CONFIG_NAME = "navigation_ippo"
CHECKPOINT_PATH = "/home/grad/doc/2027/spatel2/AD2C_testBed/saved_models/snd06.pt"

GlobalHydra.instance().clear()

sys.argv = [
    "eval_script.py",
    f"experiment.restore_file={CHECKPOINT_PATH}",
    "experiment.evaluation_episodes=10",
    "experiment.render=True",
    "experiment.evaluation_deterministic_actions=True",
    "experiment.save_folder=null",
    "model.desired_snd=0.6",
    f"+experiment.name={unique_id}",
    f"+logger.id={unique_id}"
]

@hydra.main(version_base=None, config_path=ABS_CONFIG_PATH, config_name=CONFIG_NAME)
def eval_experiment(cfg: DictConfig) -> None:

    OmegaConf.set_struct(cfg, False)
    cfg.logger.id = unique_id

    print(f"Loading model from: {cfg.experiment.restore_file}")

    experiment = get_experiment(cfg=cfg)

    print("Model loaded. Starting Evaluation...")
    experiment._evaluation_loop()
    print("Evaluation Complete.")

    experiment.close()
    wandb.finish()

if __name__ == "__main__":
    try:
        eval_experiment()
    except SystemExit:
        pass
    except Exception as e:
        print(f"An error occurred: {e}")
        wandb.finish()

Saving original WandB init function...

--- INTERCEPTING WANDB INIT ---
Forced ID: AD2C_Eval_1764394529
-------------------------------

Initialized WandB with run id: AD2C_Eval_1764394529
Loading model from: /home/grad/doc/2027/spatel2/AD2C_testBed/saved_models/snd06.pt

Algorithm: ippo, Task: vmas/navigation


/home/grad/doc/2027/spatel2/miniconda3/envs/ad2c/lib/python3.9/site-packages/torch/nn/functional.py:1956: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")



--- INTERCEPTING WANDB INIT ---
Forced ID: AD2C_Eval_1764394529
-------------------------------

Model loaded. Starting Evaluation...


/home/grad/doc/2027/spatel2/miniconda3/envs/ad2c/lib/python3.9/site-packages/torch/nn/functional.py:1956: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
/home/grad/doc/2027/spatel2/miniconda3/envs/ad2c/lib/python3.9/site-packages/torch/nn/functional.py:1956: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


Evaluation Complete.


In [ ]:
# !wandb login --relogin b3c2b62655aa322e8ab1d1ab07287749ce03ff8

## Testing Env with different task config
---

Changing the task config to check the adaptibility of learned policy. 

In [ ]:
wandb.init = forced_wandb_init

# --- STEP 1: Define Unique ID ---
unique_id = f"AD2C_Eval_{int(time.time())}"

# --- STEP 2: Force WandB to use this exact run ---
os.environ["WANDB_MODE"] = "online"
os.environ["WANDB_RUN_ID"] = unique_id
os.environ["WANDB_NAME"] = unique_id
os.environ["WANDB_INIT_TIMEOUT"] = "300"
os.environ["WANDB_SILENT"] = "true"

# --- STEP 3: Hydra Setup ---
ABS_CONFIG_PATH = "/home/grad/doc/2027/spatel2/AD2C_testBed/AD2C/ControllingBehavioralDiversity/het_control/conf"
CONFIG_NAME = "navigation_ippo"
CHECKPOINT_PATH = "/home/grad/doc/2027/spatel2/AD2C_testBed/saved_models/snd06.pt"

GlobalHydra.instance().clear()

sys.argv = [
    "eval_script.py",
    f"experiment.restore_file={CHECKPOINT_PATH}",
    "experiment.evaluation_episodes=10",
    "experiment.render=True",
    "experiment.evaluation_deterministic_actions=True",
    "experiment.save_folder=null",
    "model.desired_snd=0.6",

    "task.agents_with_same_goal=1" ,

    f"+experiment.name={unique_id}",
    f"+logger.id={unique_id}"
]

@hydra.main(version_base=None, config_path=ABS_CONFIG_PATH, config_name=CONFIG_NAME)
def eval_experiment(cfg: DictConfig) -> None:

    OmegaConf.set_struct(cfg, False)
    cfg.logger.id = unique_id

    print(f"Loading model from: {cfg.experiment.restore_file}")

    experiment = get_experiment(cfg=cfg)

    print("Model loaded. Starting Evaluation...")
    experiment._evaluation_loop()
    print("Evaluation Complete.")

    experiment.close()
    wandb.finish()

if __name__ == "__main__":
    try:
        eval_experiment()
    except SystemExit:
        pass
    except Exception as e:
        print(f"An error occurred: {e}")
        wandb.finish()


Loading model from: /home/grad/doc/2027/spatel2/AD2C_testBed/saved_models/snd06.pt

Algorithm: ippo, Task: vmas/navigation

--- INTERCEPTING WANDB INIT ---
Forced ID: AD2C_Eval_1764394624
-------------------------------



/home/grad/doc/2027/spatel2/miniconda3/envs/ad2c/lib/python3.9/site-packages/torch/nn/functional.py:1956: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


Model loaded. Starting Evaluation...


/home/grad/doc/2027/spatel2/miniconda3/envs/ad2c/lib/python3.9/site-packages/torch/nn/functional.py:1956: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


Evaluation Complete.


## Transfer Lerning
---

Finetune the policy to make it work with other task that it could not. 

In [ ]:
wandb.init = forced_wandb_init

# --- STEP 1: Define Unique ID ---
unique_id = f"AD2C_Eval_{int(time.time())}"

# --- STEP 2: Force WandB to use this exact run ---
os.environ["WANDB_MODE"] = "online"
os.environ["WANDB_RUN_ID"] = unique_id
os.environ["WANDB_NAME"] = unique_id
os.environ["WANDB_INIT_TIMEOUT"] = "300"
os.environ["WANDB_SILENT"] = "true"


ABS_CONFIG_PATH = "/home/grad/doc/2027/spatel2/AD2C_testBed/AD2C/ControllingBehavioralDiversity/het_control/conf"
CONFIG_NAME = "navigation_ippo"
CHECKPOINT_PATH = "/home/grad/doc/2027/spatel2/AD2C_testBed/saved_models/snd06.pt"

# ==========================================
# 2. RUN LOGIC
# ==========================================
new_max_frames = 18000000 
desired_snd = 0.6

GlobalHydra.instance().clear()

sys.argv = [
    "run_script.py",
    f"model.desired_snd={desired_snd}",
    f"experiment.restore_file={CHECKPOINT_PATH}",
    f"experiment.max_n_frames={new_max_frames}",
    
    # --- TASK CONFIGURATION ---
    "task.agents_with_same_goal=3", 
    "experiment.save_folder=null"
]

@hydra.main(version_base=None, config_path=ABS_CONFIG_PATH, config_name=CONFIG_NAME)
def hydra_experiment(cfg: DictConfig) -> None:
    print(f"Resuming with SND: {cfg.model.desired_snd}")
    print(f"Agents sharing a goal: {cfg.task.agents_with_same_goal}")
    
    experiment = get_experiment(cfg=cfg)
    experiment.run()

if __name__ == "__main__":
    try:
        hydra_experiment()
    except SystemExit:
        print("Experiment finished successfully.")
    except Exception as e:
        print(f"An error occurred: {e}")

Resuming with SND: 0.6
Agents sharing a goal: 3

Algorithm: ippo, Task: vmas/navigation


/home/grad/doc/2027/spatel2/miniconda3/envs/ad2c/lib/python3.9/site-packages/torch/nn/functional.py:1956: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")



--- INTERCEPTING WANDB INIT ---
Forced ID: AD2C_Eval_1764406651
-------------------------------


✅ SUCCESS: Extremum Seeking Controller initialized for group 'agents'.


 67%|██████▋   | 200/300 [00:00<?, ?it/s]/home/grad/doc/2027/spatel2/miniconda3/envs/ad2c/lib/python3.9/site-packages/torch/nn/functional.py:1956: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


[ESC] Updated SND: 0.6000000238418579 (Reward: 1.148, Update Step: -0.0000)


...

mean return = 0.007387502584606409:  99%|█████████▉| 298/300 [1:08:15<01:23, 41.57s/it]/home/grad/doc/2027/spatel2/miniconda3/envs/ad2c/lib/python3.9/site-packages/torch/nn/functional.py:1956: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


[ESC] Updated SND: 0.6705653071403503 (Reward: -3.480, Update Step: -0.2051)


mean return = -1.3856338262557983: 100%|█████████▉| 299/300 [1:08:56<00:41, 41.39s/it] /home/grad/doc/2027/spatel2/miniconda3/envs/ad2c/lib/python3.9/site-packages/torch/nn/functional.py:1956: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


[ESC] Updated SND: 0.5546724796295166 (Reward: -1.179, Update Step: -0.1159)


mean return = -1.5155255794525146: 100%|██████████| 300/300 [1:09:40<00:00, 41.80s/it]
